<a href="https://colab.research.google.com/github/123gamal/Shop_Lifting_detection_using_R2Plus1D_Finetuned/blob/main/shop_lifting_pretrained_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gdown

In [2]:
!gdown 1KCKfyIGbQi8a7bIYta3LM8dFStxVzVX-

Downloading...
From (original): https://drive.google.com/uc?id=1KCKfyIGbQi8a7bIYta3LM8dFStxVzVX-
From (redirected): https://drive.google.com/uc?id=1KCKfyIGbQi8a7bIYta3LM8dFStxVzVX-&confirm=t&uuid=74863b82-ebb8-4d57-b282-f94804de2f88
To: /content/Shop DataSet.zip
100% 1.78G/1.78G [00:19<00:00, 89.4MB/s]


In [5]:
!unzip Shop_DataSet.zip -d dataset/

Archive:  Shop_DataSet.zip
   creating: dataset/Shop DataSet/non shop lifters/
  inflating: dataset/Shop DataSet/non shop lifters/shop_lifter_n_0.mp4  
  inflating: dataset/Shop DataSet/non shop lifters/shop_lifter_n_0_1.mp4  
  inflating: dataset/Shop DataSet/non shop lifters/shop_lifter_n_1.mp4  
  inflating: dataset/Shop DataSet/non shop lifters/shop_lifter_n_1_1.mp4  
  inflating: dataset/Shop DataSet/non shop lifters/shop_lifter_n_10.mp4  
  inflating: dataset/Shop DataSet/non shop lifters/shop_lifter_n_10_1.mp4  
  inflating: dataset/Shop DataSet/non shop lifters/shop_lifter_n_100.mp4  
  inflating: dataset/Shop DataSet/non shop lifters/shop_lifter_n_100_1.mp4  
  inflating: dataset/Shop DataSet/non shop lifters/shop_lifter_n_101.mp4  
  inflating: dataset/Shop DataSet/non shop lifters/shop_lifter_n_101_1.mp4  
  inflating: dataset/Shop DataSet/non shop lifters/shop_lifter_n_102.mp4  
  inflating: dataset/Shop DataSet/non shop lifters/shop_lifter_n_102_1.mp4  
  inflating: datase

In [4]:
!pip install torch torchvision tqdm opencv-python

In [6]:
import os, random, cv2
import numpy as np
from glob import glob
from collections import Counter
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from torchvision.models.video import r2plus1d_18

In [7]:
DATA_ROOT = "/content/dataset/Shop DataSet"
CLASSES = ["non shop lifters", "shop lifters"]
NUM_CLASSES = len(CLASSES)
NUM_FRAMES = 16
FRAME_SIZE = 112
BATCH_SIZE = 8
NUM_EPOCHS = 20
LR = 1e-4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

In [8]:
def list_videos(data_root, classes):
    samples = []
    for idx, cls in enumerate(classes):
        folder = os.path.join(data_root, cls)
        for ext in ("*.mp4", "*.avi", "*.mov", "*.mkv"):
            for p in glob(os.path.join(folder, ext)):
                samples.append((p, idx))
    return samples


def read_video_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    success, frame = cap.read()
    while success:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame)
        success, frame = cap.read()
    cap.release()
    return frames


def uniform_temporal_sample(frames, num_frames):
    L = len(frames)
    if L == 0:
        return [np.zeros((FRAME_SIZE, FRAME_SIZE, 3), dtype=np.uint8)] * num_frames
    if L >= num_frames:
        idx = np.linspace(0, L - 1, num_frames, dtype=int)
        return [frames[i] for i in idx]
    else:
        idx = list(range(L)) + [i % L for i in range(num_frames - L)]
        return [frames[i] for i in idx]


# Pretrained model normalization
train_transform = T.Compose([
    T.ToPILImage(),
    T.RandomResizedCrop(FRAME_SIZE, scale=(0.8, 1.0)),
    T.RandomHorizontalFlip(),
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    T.ToTensor(),
    T.Normalize(mean=[0.43216, 0.394666, 0.37645],
                std=[0.22803, 0.22145, 0.216989])
])

val_transform = T.Compose([
    T.ToPILImage(),
    T.Resize((FRAME_SIZE, FRAME_SIZE)),
    T.ToTensor(),
    T.Normalize(mean=[0.43216, 0.394666, 0.37645],
                std=[0.22803, 0.22145, 0.216989])
])

In [9]:
class VideoDataset(Dataset):
    def __init__(self, samples, num_frames, transform, train=True):
        self.samples = samples
        self.num_frames = num_frames
        self.transform = transform
        self.train = train

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        frames = read_video_frames(path)
        frames = uniform_temporal_sample(frames, self.num_frames)
        frame_tensors = [self.transform(f) for f in frames]
        video_tensor = torch.stack(frame_tensors)
        return video_tensor, torch.tensor(label), path

In [10]:
class R2Plus1D_Finetune(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.model = r2plus1d_18(weights="KINETICS400_V1")
        in_features = self.model.fc.in_features
        self.model.fc = nn.Linear(in_features, num_classes)

    def forward(self, x):
        x = x.permute(0, 2, 1, 3, 4)  # [B, T, C, H, W] -> [B, C, T, H, W]
        return self.model(x)

In [11]:
def accuracy(preds, labels):
    preds = torch.argmax(preds, dim=1)
    return (preds == labels).float().mean().item()


def train_epoch(model, loader, criterion, optimizer):
    model.train()
    total_loss, total_acc = 0, 0
    for videos, labels, _ in tqdm(loader, desc="Train"):
        videos, labels = videos.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(videos)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        total_acc += accuracy(outputs, labels)
    return total_loss / len(loader), total_acc / len(loader)


def eval_epoch(model, loader, criterion):
    model.eval()
    total_loss, total_acc = 0, 0
    with torch.no_grad():
        for videos, labels, _ in tqdm(loader, desc="Val"):
            videos, labels = videos.to(DEVICE), labels.to(DEVICE)
            outputs = model(videos)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            total_acc += accuracy(outputs, labels)
    return total_loss / len(loader), total_acc / len(loader)

In [12]:
samples = list_videos(DATA_ROOT, CLASSES)
random.shuffle(samples)
n_train = int(0.8 * len(samples))
train_samples = samples[:n_train]
val_samples = samples[n_train:]

train_ds = VideoDataset(train_samples, NUM_FRAMES, train_transform, train=True)
val_ds = VideoDataset(val_samples, NUM_FRAMES, val_transform, train=False)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

print(f"Total videos: {len(samples)}, Train: {len(train_ds)}, Val: {len(val_ds)}")
print("Class distribution:", Counter([y for _, y in samples]))

model = R2Plus1D_Finetune(NUM_CLASSES).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=LR)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

best_acc = 0.0
save_path = "/content/best_r2plus1d.pth"

for epoch in range(1, NUM_EPOCHS + 1):
    print(f"\nEpoch {epoch}/{NUM_EPOCHS}")
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer)
    val_loss, val_acc = eval_epoch(model, val_loader, criterion)
    scheduler.step()

    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"Val   Loss: {val_loss:.4f} | Val   Acc: {val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), save_path)
        print("✅ Saved best model")

print(f"\nTraining complete. Best Val Acc = {best_acc:.4f}")


Total videos: 855, Train: 684, Val: 171
Class distribution: Counter({0: 531, 1: 324})
Downloading: "https://download.pytorch.org/models/r2plus1d_18-91a641e6.pth" to /root/.cache/torch/hub/checkpoints/r2plus1d_18-91a641e6.pth


100%|██████████| 120M/120M [00:00<00:00, 161MB/s]



Epoch 1/20


Val: 100%|██████████| 22/22 [01:31<00:00,  4.16s/it]


Train Loss: 0.3327 | Train Acc: 0.8474
Val   Loss: 0.3915 | Val   Acc: 0.9091
✅ Saved best model

Epoch 2/20


...

Train Loss: 0.0018 | Train Acc: 1.0000
Val   Loss: 0.5708 | Val   Acc: 0.6326

Epoch 19/20


Val: 100%|██████████| 22/22 [01:25<00:00,  3.87s/it]


Train Loss: 0.0013 | Train Acc: 1.0000
Val   Loss: 0.5620 | Val   Acc: 0.6326

Epoch 20/20


Val: 100%|██████████| 22/22 [01:31<00:00,  4.14s/it]

Train Loss: 0.0022 | Train Acc: 1.0000
Val   Loss: 0.6481 | Val   Acc: 0.5303

Training complete. Best Val Acc = 0.9886


In [15]:
from google.colab import drive
drive.mount('/content/drive')

!cp /content/best_r2plus1d.pth /content/drive/MyDrive/
print("✅ Best model saved to Google Drive")

Mounted at /content/drive
✅ Best model saved to Google Drive


In [ ]:
model.save("shoplifting_classifier.h5")

In [18]:
def predict_video(model, video_path, transform, num_frames):
    model.eval()
    frames = read_video_frames(video_path)
    frames = uniform_temporal_sample(frames, num_frames)
    frame_tensors = [transform(f) for f in frames]
    video_tensor = torch.stack(frame_tensors).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        logits = model(video_tensor)
        probs = torch.softmax(logits, dim=1).cpu().numpy()[0]
        pred = int(np.argmax(probs))
    return CLASSES[pred], probs


# Example usage after training:
model.load_state_dict(torch.load(save_path))
label, probs = predict_video(model, "/content/dataset/Shop DataSet/non shop lifters/shop_lifter_n_0_1.mp4", val_transform, NUM_FRAMES)
print("Prediction:", label, "| Probabilities:", probs)


Prediction: non shop lifters | Probabilities: [0.827701   0.17229901]
